## Downloading the word embeddings

In [1]:
import numpy as np
!pip3 install numpy --upgrade

!pip3 install sacremoses

Requirement already up-to-date: numpy in /usr/local/lib/python3.6/dist-packages (1.18.3)
     |████████████████████████████████| 890kB 2.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.41-cp36-none-any.whl size=893334 sha256=5dab642d4ab02d16dc3eecf962beb6e23d2bf494135bd41f6f5c3b66cabf70f0
  Stored in directory: /root/.cache/pip/wheels/22/5a/d4/b020a81249de7dc63758a34222feaa668dbe8ebfe9170cc9b1
Successfully built sacremoses


In [2]:
!pip3 install transformers

     |████████████████████████████████| 573kB 2.8MB/s 
     |████████████████████████████████| 3.7MB 45kB/s 
     |████████████████████████████████| 1.0MB 69.0MB/s 


In [3]:
!pip3 install bert-embedding
!pip3 install allennlp
!pip3 install pytorch_transformers

     |████████████████████████████████| 13.8MB 239kB/s 
     |████████████████████████████████| 29.6MB 104kB/s 
     |████████████████████████████████| 215kB 67.5MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.6.0-cp36-none-any.whl size=259916 sha256=8c62f88fc9db2a543408dcf6ea92c9e6b7268ae09d1e4453e10999a14d86d3fc
  Stored in directory: /root/.cache/pip/wheels/ff/48/ac/a77c79aa416ba6dd7bf487f2280b0471034f66141617965914
Successfully built gluonnlp
ERROR: xarray 0.15.1 has requirement numpy>=1.15, but you'll have numpy 1.14.6 which is incompatible.
ERROR: umap-learn 0.4.1 has requirement numpy>=1.15, but you'll have numpy 1.14.6 which is incompatible.
ERROR: tensorflow 2.2.0rc3 has requirement numpy<2.0,>=1.16.0, but you'll have numpy 1.14.6 which is incompatible.
ERROR: spacy 2.2.4 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
ERROR: plotnine 0.6.0 has requirement numpy>=1.16.0, but you'll have numpy 1.14.6 which is incompatible.
ERROR: numba

     |████████████████████████████████| 7.6MB 125kB/s 
     |████████████████████████████████| 163kB 55.8MB/s 
     |████████████████████████████████| 30.9MB 101kB/s 
     |████████████████████████████████| 204kB 73.8MB/s 
     |████████████████████████████████| 5.2MB 59.9MB/s 
     |████████████████████████████████| 61kB 9.5MB/s 
     |████████████████████████████████| 133kB 62.7MB/s 
     |████████████████████████████████| 51kB 8.3MB/s 
     |████████████████████████████████| 245kB 68.2MB/s 
     |████████████████████████████████| 256kB 62.8MB/s 
     |████████████████████████████████| 3.2MB 74.7MB/s 
     |████████████████████████████████| 92kB 12.5MB/s 
     |████████████████████████████████| 2.1MB 75.6MB/s 
     |████████████████████████████████| 51kB 8.5MB/s 
  Created wheel for overrides: filename=overrides-2.8.0-cp36-none-any.whl size=5609 sha256=c6fd67d8dce42145de6812cb4a57a33bf475fe26e8d9b1d647afc2976fe146c2
  Stored in directory: /root/.cache/pip/wheels/df/f1/ba/eaf6cd7d284d

## Importing libraries

In [4]:
!pip install --upgrade numpy
import scipy

import nltk
nltk.download('punkt')

import tarfile

from nltk.tokenize import word_tokenize
from bert_embedding import BertEmbedding
from allennlp.commands.elmo import ElmoEmbedder

import torch
from pytorch_transformers import *

import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

     |████████████████████████████████| 20.2MB 1.3MB/s 
ERROR: mxnet 1.4.0 has requirement numpy<1.15.0,>=1.8.2, but you'll have numpy 1.18.3 which is incompatible.
ERROR: en-core-web-sm 2.2.5 has requirement spacy>=2.2.2, but you'll have spacy 2.1.9 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: bert-embedding 1.0.1 has requirement numpy==1.14.6, but you'll have numpy 1.18.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.14.6
    Uninstalling numpy-1.14.6:
      Successfully uninstalled numpy-1.14.6


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Preprocessing:
* Tokenize sentences
* Assigning corresponding word embeddings
* Create cosine similarity matrix based on the distances between the word embeddings.

In [0]:
class Embeddings:
  
  def __init__(self, sentence_1, sentence_2):
    self.sentence_1 = sentence_1
    self.sentence_2 = sentence_2
    self.tokenized_sent_1 = word_tokenize(self.sentence_1)
    self.tokenized_sent_2 = word_tokenize(self.sentence_2)
    
  def get_elmo_embedding(self):
    '''Creates ELMo word embeddings for the given words
    param: list, list
    returns: ndarray, ndarray
      Returns the ELMo embeddings of the tokens of two sentences'''
    
    elmo = ElmoEmbedder()
    
    elmo_embedding_1 = elmo.embed_sentence(self.tokenized_sent_1)
    elmo_embedding_2 = elmo.embed_sentence(self.tokenized_sent_2)

    return elmo_embedding_1, elmo_embedding_2
    
  def get_bert_embedding(self):
    '''Creates word embeddings taken from BERT language representation
    returns: list, list
      Returns the BERT embeddings of the tokens of two sentences'''
    
    bert_embedding_1  = BertEmbedding().embedding(sentences = self.tokenized_sent_1)
    bert_embedding_2  = BertEmbedding().embedding(sentences = self.tokenized_sent_2)
    
    return bert_embedding_1, bert_embedding_2
    
  def get_gpt_embedding(self):
    '''Creates word embeddings of GPT
    returns: tensor
      Returns the GPT embeddings of the tokens of two sentences'''
    
    # model= torch.hub.load('huggingface/pytorch-pretrained-BERT', 'openAIGPTModel', 'openai-gpt')
    # tokenizer=torch.hub.load('huggingface/pytorch-pretrained-BERT', 'openAIGPTTokenizer', 'openai-gpt')

    tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'openai-gpt')
    model = torch.hub.load('huggingface/pytorch-transformers', 'modelWithLMHead', 'openai-gpt')

    indexed_token_1 = tokenizer.convert_tokens_to_ids(self.tokenized_sent_1)
    indexed_token_2 = tokenizer.convert_tokens_to_ids(self.tokenized_sent_2)
    
    tokens_tensor_1 = torch.tensor([indexed_token_1])
    tokens_tensor_2 = torch.tensor([indexed_token_2])

    gpt_embedding_1 = model(tokens_tensor_1)
    gpt_embedding_2 = model(tokens_tensor_2)

    return gpt_embedding_1, gpt_embedding_2

  def get_gpt2_embedding(self):
    '''Creates word embeddings of GPT
    returns: tensor
      Returns the GPT2 embeddings of the tokens of two sentences'''
    tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'gpt2')
    model = torch.hub.load('huggingface/pytorch-transformers', 'modelWithLMHead', 'gpt2')

    indexed_token_1 = tokenizer.convert_tokens_to_ids(self.tokenized_sent_1)
    indexed_token_2 = tokenizer.convert_tokens_to_ids(self.tokenized_sent_2)
    
    tokens_tensor_1 = torch.tensor([indexed_token_1])
    tokens_tensor_2 = torch.tensor([indexed_token_2])

    gpt2_embedding_1 = model(tokens_tensor_1)
    gpt2_embedding_2 = model(tokens_tensor_2)

    return gpt2_embedding_1, gpt2_embedding_2

### Converting Embeddings to list 

In [0]:
class Embedding2Array:
  def elmo(self,embedding):
    ''' Creates the list of arrays of each corresponding word
    parameters
    embedding : ndarray
    returns: list
      Returns the list of elmo embedding of each word 
    '''
    word_array = []
    for i in range(len(embedding[2])):
      word_array.append(embedding[2][i])
    return word_array
    
  def bert(self,embedding):
    ''' Creates the list of arrays of each corresponding word
    param
    embedding: list
    returns: list
      Returns the list of BERT embedding of each word 
    '''
    word_array = []
    for i in range(len(embedding)):
      word_array.append(embedding[i][1][0])
    return word_array
  
  def gpt(self,embedding):
    ''' Creates the list of arrays of each corresponding word
    param
    embedding: tensor
    returns: list
      Returns the list of GPT embedding of each word 
    '''
    word_array = []

    for i in range(embedding[0].shape[1]):
      word_array.append(embedding[0][0][i].tolist())

    return word_array
  
  def gpt2(self, embedding):
    ''' Creates the list of arrays of each corresponding word
    param
    embedding: tensor
    returns: list
      Returns the list of GPT2 embedding of each word 
    '''
    word_array=[]

    for i in range(embedding[0].size()[1]):
      word_array.append(embedding[0][0][i].tolist())

    return word_array


 ## Processing:
 * Creating a cosine similarity matrix
 * Plotting the similarity matrix based on the cosine matrix from preprocessing class
 * Creating a list of similar words of one sentence for each word in the other sentence.

In [0]:
class Main:
  def __init__(self, sentence_1, sentence_2):
    self.first_sentence = sentence_1
    self.second_sentence = sentence_2

  def elmo(self):
    ''' Creates a list of ELMo word embeddings of each sentence
    returns: list, list
      The list of embeddings of words of each sentence
    '''

    embed = Embeddings(self.first_sentence, self.second_sentence)
    embed2array = Embedding2Array()
    first_embedding, second_embedding = embed.get_elmo_embedding()
    first_array, second_array = embed2array.elmo(first_embedding), embed2array.elmo(second_embedding)
    return first_array, second_array

  def bert(self):
    '''Creates a list of BERT word embeddings of each sentence
    returns: list, list
      The list of embeddings of words of each sentence
    '''
    embed = Embeddings(self.first_sentence, self.second_sentence)
    embed2array = Embedding2Array()
    first_embedding, second_embedding = embed.get_bert_embedding()
    first_array, second_array = embed2array.bert(first_embedding), embed2array.bert(second_embedding)
    return first_array, second_array

  def gpt(self):
    '''Creates a list of GPT word embeddings of each sentence
    returns: list, list
      The list of embeddings of words of each sentence
    '''
    embed = Embeddings(self.first_sentence, self.second_sentence)
    embed2array = Embedding2Array()
    first_embedding, second_embedding = embed.get_gpt_embedding()
    first_array, second_array = embed2array.gpt(first_embedding), embed2array.gpt(second_embedding)
    return first_array, second_array

  def gpt2(self):
    '''Creates a list of GPT word embeddings of each sentence
    returns: list, list
      The list of embeddings of words of each sentence
    '''
    embed = Embeddings(self.first_sentence, self.second_sentence)
    embed2array = Embedding2Array()
    first_embedding, second_embedding = embed.get_gpt2_embedding()
    first_array, second_array = embed2array.gpt2(first_embedding), embed2array.gpt2(second_embedding)
    return first_array, second_array

--------------------------------------------------------------------------------

In [0]:
class SimilarityFunctions:

  def __init__(self, sentence_1, sentence_2, word_array_1, word_array_2):
    self.sentence_1 = sentence_1
    self.sentence_2 = sentence_2
    self.word_array_1 = word_array_1
    self.word_array_2 = word_array_2
  
  def cosine_similarity_matrix(self):
    '''
    Creates a matrix depicting the cosine distances between the words of two sentences
    returns: array
      Similarity matrix of words in two sentences
    '''
    matrix = np.zeros((len(self.word_array_1), len(self.word_array_2)))

    for i in range(0, len(self.word_array_1)):
      for j in range(0, len(self.word_array_2)):
        matrix[i][j]= scipy.spatial.distance.cosine(self.word_array_1[i], self.word_array_2[j])
    return matrix.T
  
  def plot_similarity_matrix(self, title):
    '''Plot the similarity matrix of two sentences
    param:
    title: str
      Labels the plot with the corresponding title
    returns: None
    '''
    x_labels, y_labels = word_tokenize(self.sentence_1), word_tokenize(self.sentence_2)
    similarity_matrix = self.cosine_similarity_matrix()
    sns.heatmap(similarity_matrix, vmin=0, vmax=1, xticklabels=x_labels, yticklabels=y_labels, cmap="YlGnBu", annot=True)
    plt.title(title)
    plt.show()    
  
  def get_similar_words(self):
    '''Prints similar word from second sentence for each word in the first sentence
    returns: list of similar words
    '''

    token_1 = word_tokenize(self.sentence_1)
    token_2 = word_tokenize(self.sentence_2)
    
    similarity_matrix = self.cosine_similarity_matrix()
    
    similar_word_dict = {}
    for row in range(0,len(similarity_matrix[0])):
      
      min_val = min(similarity_matrix.T[row]) #Here min value of transpose is found. To understand it print similarity matrix and find the logic
      index = (np.where(similarity_matrix.T[row] == min_val))[0]
      similar_word_list = []
      
      for i in range(0,len(index)):
        similar_word_list.append(token_2[index[i]])
      similar_word_dict[token_1[row]] = similar_word_list
      
    print('Similar words in two sentences are :', similar_word_dict)

In [0]:
class AnalyzeSentenceEmbeddings:
  def __init__(self,sentence_1, sentence_2):
    self.sentence_1 = sentence_1
    self.sentence_2 = sentence_2
    
  def bert(self):
    '''Plots similarity matrix using BERT embeddings and returns similar words in both the sentences
    returns: dict
      A dictionary matching the similar words from one sentence to other using BERT embeddings
    '''    
    main = Main(self.sentence_1, self.sentence_2)
    array_1, array_2 = main.bert()

    sentence_embedding_1 = sum(array_1)
    sentence_embedding_2 = sum(array_2)

    distance = scipy.spatial.distance.cosine(sentence_embedding_1, sentence_embedding_2)
    return 1-distance
    
  
  def elmo(self):
    '''Plots similarity matrix using ELMo embeddings and returns similar words in both the sentences
    returns: dict
      A dictionary matching the similar words from one sentence to other using ELMo embeddings
    '''      
    main = Main(self.sentence_1, self.sentence_2)
    array_1, array_2 = main.elmo()
    
    sentence_embedding_1 = sum(array_1)
    sentence_embedding_2 = sum(array_2)

    distance = scipy.spatial.distance.cosine(sentence_embedding_1, sentence_embedding_2)
    return 1-distance

  
  def gpt(self):
    '''Plots similarity matrix using GPT embeddings and returns similar words in both the sentences
    returns: dict
      A dictionary matching the similar words from one sentence to other using GPT embeddings
    '''    
    main = Main(self.sentence_1, self.sentence_2)
    array_1, array_2 = main.gpt()

    sentence_embedding_1 = [sum(i) for i in zip(*array_1)]
    sentence_embedding_2 = [sum(i) for i in zip(*array_2)]

    print(array_1, sentence_embedding_1)

    distance = scipy.spatial.distance.cosine(sentence_embedding_1, sentence_embedding_2)
    return 1-distance
  

  def gpt2(self):
    '''Plots similarity matrix using GPT2 embeddings and returns similar words in both the sentences
    returns: dict
      A dictionary matching the similar words from one sentence to other using GPT embeddings
    '''    
    main = Main(self.sentence_1, self.sentence_2)
    array_1, array_2 = main.gpt2()

    sentence_embedding_1 = [sum(i) for i in zip(*array_1)]
    sentence_embedding_2 = [sum(i) for i in zip(*array_2)]

    distance = scipy.spatial.distance.cosine(sentence_embedding_1, sentence_embedding_2)
    return 1-distance


In [10]:
!jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000000

/usr/local/lib/python2.7/dist-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")
Traceback (most recent call last):
  File "/usr/local/bin/jupyter-notebook", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python2.7/dist-packages/jupyter_core/application.py", line 267, in launch_instance
    return super(JupyterApp, cls).launch_instance(argv=argv, **kwargs)
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 657, in launch_instance
    app.initialize(argv)
  File "</usr/local/lib/python2.7/dist-packages/decorator.pyc:decorator-gen-7>", line 2, in initialize
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 87, in catch_config_error
    return method(app, *args, **kwargs)
  File "/usr/local/lib/python2.7/dist-packages/notebook/notebookapp.py", line 1368, in 

In [0]:
import json

In [12]:
sentences = ['To find problem and errors in a program before it is finalized', 'A program that simulates the behavior of portions of the desired software product.', 'Existing classes can be reused, and program maintenance and verification are easier.','Coding and Debugging programs are usually easier, as either the compiler will specify the object that is working incorrectly, or the function an object was assigned to will be easier to identify.  The code itself also looks more organized and is easier to read and will help to avoid redundant coding.  Post-programming maintenance is also easier.  Also, modules can be reused several times in other programs without too much hassle.   Abstraction is the art of breaking down one big problem into smaller, simpler problems and solving them.  Many of the smaller problems are shared between unique complex problems, and creating modules to these solve these smaller repeated problems can save time when you encounter them again.']
sentences_wo = ['find problem errors program before finalized','program simulates behavior portions desired software product', 'Existing classes reused program maintenance verification easier','Coding Debugging programs easier, compiler specify object working incorrectly function object assigned easier identify code  looks more organized easier read help avoid redundant coding Post-programming maintenance easier modules reused several times programs hassle Abstraction art breaking down big problem smaller simpler problems solving them Many smaller problems shared unique complex problems creating modules solve smaller repeated problems save time encounter again']
sentences_sp = ['To find problems and erorrs in a programm before it is finalized', 'A programme that simulates the behaviour of portions of the desired softwear product.', 'Existing clasess can be reused, and program maintanance and verification are easier.','Coding and Debuging programs are usualy easier, as ither the compiler will specsify the object that is working in correctly, or the function an object was assigned to will be easier to identify.  The kode itself also looks more organized and is easier to read and will help to avoid redundent coding.  Post-programming maintenence is also easier.  Also, modules can be reused several times in other programs without too much hazzle.   Abstraction is the art of breaking down one big problem into smaller, simpler problems and solving them.  Many of the smaller probllems are shared between unik complex problems, and creating modules to these solve these smaller repated problems can save time when you ancounter them egain.']
results_dict = dict()
count = 1

for sent_pair in zip(sentences, sentences_wo):
  print(sent_pair)
  analyze_sentence = AnalyzeSentenceEmbeddings(sent_pair[0], sent_pair[1])
  results_dict['elmo_'+str(count)] = analyze_sentence.elmo()
  results_dict['bert_'+str(count)] = analyze_sentence.bert()
  results_dict['gpt_'+str(count)] = analyze_sentence.gpt()
  results_dict['gpt2_'+str(count)] = analyze_sentence.gpt2()
  count += 1

for sent_pair in zip(sentences, sentences_sp):
  print(sent_pair)
  analyze_sentence = AnalyzeSentenceEmbeddings(sent_pair[0], sent_pair[1])
  results_dict['elmo_'+str(count)] = analyze_sentence.elmo()
  results_dict['bert_'+str(count)] = analyze_sentence.bert()
  results_dict['gpt_'+str(count)] = analyze_sentence.gpt()
  results_dict['gpt2_'+str(count)] = analyze_sentence.gpt2()
  count += 1


('To find problem and errors in a program before it is finalized', 'find problem errors program before finalized')


100%|██████████| 374434792/374434792 [00:19<00:00, 19444620.48B/s]


Vocab file is not found. Downloading.


Downloading: "https://github.com/huggingface/pytorch-transformers/archive/master.zip" to /root/.cache/torch/hub/master.zip


Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_master


...

Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_master
Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_master
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_master
Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_master


('Existing classes can be reused, and program maintenance and verification are easier.', 'Existing clasess can be reused, and program maintanance and verification are easier.')


Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_master
Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_master
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_master
Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_master


('Coding and Debugging programs are usually easier, as either the compiler will specify the object that is working incorrectly, or the function an object was assigned to will be easier to identify.  The code itself also looks more organized and is easier to read and will help to avoid redundant coding.  Post-programming maintenance is also easier.  Also, modules can be reused several times in other programs without too much hassle.   Abstraction is the art of breaking down one big problem into smaller, simpler problems and solving them.  Many of the smaller problems are shared between unique complex problems, and creating modules to these solve these smaller repeated problems can save time when you encounter them again.', 'Coding and Debuging programs are usualy easier, as ither the compiler will specsify the object that is working in correctly, or the function an object was assigned to will be easier to identify.  The kode itself also looks more organized and is easier to read and wil

Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_master
Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_master
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_master
Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_master


In [13]:
print(len(sentences_wo))

4


In [0]:
with open('/content/sample_data/sentence_embed_sim.json','w') as json_file:
  json.dump(results_dict,json_file)

## Dataset


--------------------------------------------------------------------------------

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [16]:
class GetSimilarityList:
   
  def __init__(self):
    self.mohler_data_path = "/content/drive/My Drive/Colab Notebooks/RnD/dataset/datasets/mohler/cleaned"
    self.answers = pd.read_csv(self.mohler_data_path+'/answers.csv', sep = '\t', index_col = 0)
    self.questions = pd.read_csv(self.mohler_data_path+'/questions.csv', sep = '\t', index_col = 0)
    self.answers_dict = pd.Series(self.answers.id.values,index=self.answers.answer).to_dict()
   
  def create_bert_similarity(self):

    bert_similarity_dict = {}

    for stu_answer in self.answers_dict:
      answer_id = self.answers_dict[stu_answer]
      expected_answer = self.questions.loc[self.questions['id'] == answer_id, 'solution'].iloc[0]
      print("Bert:", expected_answer)
      get_similarity_score = AnalyzeSentenceEmbeddings(stu_answer, expected_answer)
      bert_similarity_dict[stu_answer] = get_similarity_score.bert()
      print(bert_similarity_dict)     
    return bert_similarity_dict

  def create_elmo_similarity(self):

    elmo_similarity_dict = {}

    for stu_answer in self.answers_dict:
      answer_id = self.answers_dict[stu_answer]
      expected_answer = self.questions.loc[self.questions['id'] == answer_id, 'solution'].iloc[0]
      print(expected_answer)
      get_similarity_score = AnalyzeSentenceEmbeddings(stu_answer, expected_answer)
      elmo_similarity_dict[stu_answer] = get_similarity_score.elmo()
     
    return elmo_similarity_dict

  def create_gpt_similarity(self):

    gpt_similarity_dict = {}

    for stu_answer in self.answers_dict:
      answer_id = self.answers_dict[stu_answer]
      expected_answer = self.questions.loc[self.questions['id'] == answer_id, 'solution'].iloc[0]
      print(expected_answer)
      get_similarity_score = AnalyzeSentenceEmbeddings(stu_answer, expected_answer)
      gpt_similarity_dict[stu_answer] = get_similarity_score.gpt()
     
    return gpt_similarity_dict

  def create_gpt2_similarity(self):

    gpt2_similarity_dict = {}

    for stu_answer in self.answers_dict:
      answer_id = self.answers_dict[stu_answer]
      expected_answer = self.questions.loc[self.questions['id'] == answer_id, 'solution'].iloc[0]
      print(expected_answer)
      get_similarity_score = AnalyzeSentenceEmbeddings(stu_answer, expected_answer)
      gpt2_similarity_dict[stu_answer] = get_similarity_score.gpt2()
     
    return gpt2_similarity_dict

get_similarity_dict = GetSimilarityList()
bert_simscore_dict = get_similarity_dict.create_bert_similarity()
elmo_simscore_dict = get_similarity_dict.create_elmo_similarity()
gpt_simscore_dict = get_similarity_dict.create_gpt_similarity()
gpt2_simscore_dict = get_similarity_dict.create_gpt2_similarity()

Bert: To simulate the behaviour of portions of the desired software product. 
{'High risk problems are address in the prototype program to make sure that the program is feasible.  A prototype may also be used to show a company that the software can be possibly programmed.  ': 0.9723823070526123}
Bert: To simulate the behaviour of portions of the desired software product. 
{'High risk problems are address in the prototype program to make sure that the program is feasible.  A prototype may also be used to show a company that the software can be possibly programmed.  ': 0.9723823070526123, 'To simulate portions of the desired final product with a quick and easy program that does a small specific job. It is a way to help see what the problem is and how you may solve it in the final project.': 0.9716012477874756}
Bert: To simulate the behaviour of portions of the desired software product. 
{'High risk problems are address in the prototype program to make sure that the program is feasible.  

KeyboardInterrupt: ignored

In [0]:
# Finding average number of answers
total_ans = 0
for answer_id in answers.id.unique():
  total_ans = total_ans + len(answers[answers.id == answer_id])

print(total_ans/len(answers.id.unique()))

## Adding similarity scores to the csv files

In [0]:
with open("/content/drive/My Drive/Colab Notebooks/RnD/dataset/similarity_scores/bert/bert_similarity_score.json", "rt") as json_file:
  bert_simscore_dict = json.load(json_file)

In [0]:
mohler_data_path = "/content/drive/My Drive/Colab Notebooks/RnD/dataset/datasets/mohler/cleaned"
answers = pd.read_csv(mohler_data_path+'/answers.csv', sep = '\t', index_col = 0)
questions =pd.read_csv(mohler_data_path+'/questions.csv', sep = '\t', index_col = 0)


answers['bert_similarity_score'] = answers["answer"].map(bert_simscore_dict)

answers.to_csv("/content/drive/My Drive/Colab Notebooks/RnD/dataset/datasets/mohler/cleaned/answers_with_bert_similarity.csv")

## Creating random training data for regression


In [17]:
!pip install mord

  Created wheel for mord: filename=mord-0.6-cp36-none-any.whl size=6008 sha256=06b1547e487084e72492d47d2b7c0176a42cdb702051ef4a9591812a9cc9fc07
  Stored in directory: /root/.cache/pip/wheels/98/14/b2/244c2cec93a0c6edb29b488bd6b2710ded7e9d457033b86366
Successfully built mord


In [0]:

from mord import OrdinalRidge
from math import sqrt

import sklearn
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

import pandas as pd
import numpy as np
import scipy 
from scipy.stats import spearmanr

import matplotlib.pyplot as plt


### Read data

In [0]:
mohler_data_path = "/content/drive/My Drive/Colab Notebooks/RnD/dataset/datasets/mohler/cleaned"
answers_with_similarity_score = pd.read_csv(mohler_data_path+'/answers_with_similarity_score.csv', skiprows= [2273,2274,2275,2276,2277,2278,2279])

In [0]:
def train_test_split(data, percentage):

  msk = np.random.rand(len(data)) < (percentage/100)

  train_data = answers_with_similarity_score[msk]
  test_data = answers_with_similarity_score[~msk]

  return train_data, test_data

train_data, test_data = train_test_split(answers_with_similarity_score, 70)

In [21]:
train_data.columns

Index(['Unnamed: 0', 'id', 'answer', 'score_me', 'score_other', 'score_avg',
       'bert_distance', 'bert_similarity_score', 'normalized_bert_score',
       'elmo_similarity_score', 'normalized_elmo_score',
       'gpt_similarity_score', 'normalized_gpt_score', 'gpt2_similarity_score',
       'normalized_gpt2_score'],
      dtype='object')

In [0]:
class GetDataXY:
  def __init__(self,data):
    self.data = data
    self.data_y = data.score_avg.to_numpy().reshape(-1,1)
    # self.data_y = data.score_other.to_numpy().reshape(-1,1)
    # self.data_y = data.score_me.to_numpy().reshape(-1,1)
  
  def bert(self):
    data_bert_x = self.data.normalized_bert_score.to_numpy().reshape(-1,1)
    return data_bert_x, self.data_y

  def elmo(self):
    data_elmo_x = self.data.normalized_elmo_score.to_numpy().reshape(-1,1)
    return data_elmo_x, self.data_y

  def gpt(self):
    data_gpt_x = self.data.normalized_gpt_score.to_numpy().reshape(-1,1)
    return data_gpt_x, self.data_y

  def gpt2(self):
    data_gpt2_x = self.data.normalized_gpt2_score.to_numpy().reshape(-1,1)
    return data_gpt2_x, self.data_y


In [0]:
class RegressionAnalysis:
  def __init__(self, training_data, testing_data):
    self.training_data = training_data
    self.testing_data = testing_data
    self.regression_train_data = GetDataXY(self.training_data)
    self.regression_test_data = GetDataXY(self.testing_data)
    # self.clf = LinearRegression()
    self.clf = OrdinalRidge()
    # self.clf = Ridge()

  def bert(self):

    train_x, train_y = self.regression_train_data.bert()
    test_x, test_y = self.regression_test_data.bert()    
    self.clf.fit(train_x, train_y)
    test_y_predict = self.clf.predict(test_x)
    test_y_predict_rounded = np.round(test_y_predict*8)/8

    return test_y, test_y_predict_rounded

  def elmo(self):

    train_x, train_y = self.regression_train_data.elmo()
    test_x, test_y = self.regression_test_data.elmo()    
    self.clf.fit(train_x, train_y)
    test_y_predict = self.clf.predict(test_x)
    test_y_predict_rounded = np.round(test_y_predict*8)/8

    return test_y, test_y_predict_rounded

  def gpt(self):

    train_x, train_y = self.regression_train_data.gpt()
    test_x, test_y = self.regression_test_data.gpt()    
    self.clf.fit(train_x, train_y)
    test_y_predict = self.clf.predict(test_x)
    test_y_predict_rounded = np.round(test_y_predict*8)/8

    return test_y, test_y_predict_rounded
  
  def gpt2(self):

    train_x, train_y = self.regression_train_data.gpt2()
    test_x, test_y = self.regression_test_data.gpt2()    
    self.clf.fit(train_x, train_y)
    test_y_predict = self.clf.predict(test_x)
    test_y_predict_rounded = np.round(test_y_predict*8)/8

    return test_y, test_y_predict_rounded

In [0]:
plt.rc('font', family='serif', weight='bold')
plt.rc('lines', lw=2)
plt.rc('xtick', labelsize='large')
plt.rc('ytick', labelsize='large')
plt.rc('axes', linewidth=2)

In [0]:
class GetResults:
  def __init(self):
    pass
  
  def pearson_correlation(self,x,y):
        mean_x = sum(x) / len(x)
        mean_y = sum(y) / len(y)  
        cov = sum((a - mean_x) * (b - mean_y) for (a,b) in zip(x,y)) / len(x)
        
        std_x, std_y = np.std(x), np.std(y)

        p = cov/(std_x*std_y)

        return float(p)

  def plot_relation(self, x, y, col, x_label, y_label, title):
    plt.scatter(x,y,color=col)
    plt.xlabel(x_label, fontsize=14, weight='bold')
    plt.ylabel(y_label, fontsize=14, weight='bold')
    plt.grid(linestyle='--')
    plt.title(title)
    plt.show()

  def bert(self, actual_score, bert_predicted_score):
    
    self.plot_relation(actual_score, bert_predicted_score, 'blue', 'Actual avg score', 'Predicted score', 'BERT:Relation between actual avg score and predicted score')
    print('BERT(pretrained) Root mean squared error is ', sqrt(mean_squared_error(actual_score, bert_predicted_score)))
    print('BERT(pretrained) Pearson correlation is ', self.pearson_correlation(actual_score, bert_predicted_score))   
    print('BERT(pretrained) Spearman correlation is ', spearmanr(actual_score, bert_predicted_score))   

  def elmo(self, actual_score, elmo_predicted_score):

    self.plot_relation(actual_score, elmo_predicted_score, 'red', 'Actual avg score', 'Predicted score', 'ELMo:Relation between actual avg score and predicted score')    
    print('ELMo(pretrained) Root mean squared error is ', sqrt(mean_squared_error(actual_score, elmo_predicted_score)))
    print('ELMo(pretrained) Pearson correlation is ', self.pearson_correlation(actual_score, elmo_predicted_score))
    print('ELMo(pretrained) Sperman correlation is ', spearmanr(actual_score, elmo_predicted_score))

  def gpt(self, actual_score, gpt_predicted_score):
    
    self.plot_relation(actual_score, gpt_predicted_score, 'green', 'Actual avg score', 'Predicted score', 'GPT:Relation between actual avg score and predicted score')
    print('GPT(pretrained) Root mean squared error is ', sqrt(mean_squared_error(actual_score, gpt_predicted_score)))
    print('GPT(pretrained) Pearson correlation is ', self.pearson_correlation(actual_score, gpt_predicted_score))
    print('GPT(pretrained) Spearman correlation is ', spearmanr(actual_score, gpt_predicted_score))

  def gpt2(self, actual_score, gpt2_predicted_score):

    self.plot_relation(actual_score, gpt2_predicted_score, 'grey', 'Actual avg score', 'Predicted score', 'GPT2:Relation between actual avg score and predicted score')    
    print('GPT2(pretrained) Root mean squared error is ', sqrt(mean_squared_error(actual_score, gpt2_predicted_score)))
    print('GPT2(pretrained) Pearson correlation is ', self.pearson_correlation(actual_score, gpt2_predicted_score))
    print('GPT2(pretrained) Spearman correlation is ', spearmanr(actual_score, gpt2_predicted_score))

In [0]:
regression_analysis = RegressionAnalysis(train_data, test_data)
actual_avg_score, predicted_score_bert = regression_analysis.bert()
_, predicted_score_elmo = regression_analysis.elmo()
_, predicted_score_gpt = regression_analysis.gpt()
_, predicted_score_gpt2 = regression_analysis.gpt2()

In [0]:
get_results = GetResults()
get_results.elmo(actual_avg_score, predicted_score_elmo)
get_results.gpt(actual_avg_score, predicted_score_gpt)
get_results.bert(actual_avg_score, predicted_score_bert)
get_results.gpt2(actual_avg_score, predicted_score_gpt2)

In [0]:
grader_other = answers_with_similarity_score.score_other.to_list()
grader_me = answers_with_similarity_score.score_me.to_list()
grader_avg = answers_with_similarity_score.score_avg.to_list()

In [0]:
print(get_results.pearson_correlation(grader_other, grader_me))
print(get_results.pearson_correlation(grader_other, grader_avg))
print(get_results.pearson_correlation(grader_me, grader_avg))